In [279]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [281]:
data=pd.read_csv("C://Users//djuid//Downloads//Dataset//Dataset//[use case 1] Inventory Optimization for Retail//demand_forecasting.csv")

In [283]:
data.columns

Index(['Product ID', 'Date', 'Store ID', 'Sales Quantity', 'Price',
       'Promotions', 'Seasonality Factors', 'External Factors', 'Demand Trend',
       'Customer Segments'],
      dtype='object')

In [285]:
data.head()

,Product ID,Date,Store ID,Sales Quantity,Price,Promotions,Seasonality Factors,External Factors,Demand Trend,Customer Segments
0,4277,2024-01-03,48,330,24.38,No,Festival,Competitor Pricing,Increasing,Regular
1,5540,2024-04-29,10,334,74.98,Yes,Holiday,Weather,Stable,Premium
2,5406,2024-01-11,67,429,24.83,Yes,Holiday,Economic Indicator,Decreasing,Premium
3,5617,2024-04-04,17,298,13.41,No,NaN,Economic Indicator,Stable,Regular
4,3480,2024-12-14,33,344,94.96,Yes,Festival,Weather,Increasing,Regular


In [287]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

warnings.filterwarnings('ignore')

# --- Load Dataset ---
data = pd.read_csv("C://Users//djuid//Downloads//Dataset//Dataset//[use case 1] Inventory Optimization for Retail//demand_forecasting.csv")

# --- Data Preprocessing ---

# Fill missing values based on logic
data['Seasonality Factors'] = data.apply(
    lambda row: 'High Demand Season' if pd.isna(row['Seasonality Factors']) and row['Sales Quantity'] > np.percentile(data['Sales Quantity'], 75)
    else ('Normal Day' if pd.isna(row['Seasonality Factors']) else row['Seasonality Factors']),
    axis=1
)

data['External Factors'] = data.apply(
    lambda row: 'Impactful' if pd.isna(row['External Factors']) and row['Sales Quantity'] > np.percentile(data['Sales Quantity'], 75)
    else ('Regular' if pd.isna(row['External Factors']) else row['External Factors']),
    axis=1
)

# Convert 'Date' to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Lag features
data['Sales_Lag_1'] = data.groupby('Product ID')['Sales Quantity'].shift(1)
data['Sales_Lag_7'] = data.groupby('Product ID')['Sales Quantity'].shift(7)
data['Sales_Lag_30'] = data.groupby('Product ID')['Sales Quantity'].shift(30)

data['Sales_Lag_1'].fillna(0, inplace=True)
data['Sales_Lag_7'].fillna(data.groupby('Product ID')['Sales Quantity'].transform('median'), inplace=True)
data['Sales_Lag_30'].fillna(data.groupby('Product ID')['Sales Quantity'].transform('median'), inplace=True)

# Weekly and Monthly Averages
data['Sales_Weekly_Avg'] = data.groupby([data['Product ID'], pd.Grouper(key='Date', freq='W')])['Sales Quantity'].transform('mean')
data['Sales_Monthly_Avg'] = data.groupby([data['Product ID'], pd.Grouper(key='Date', freq='M')])['Sales Quantity'].transform('mean')

data['Sales_Weekly_Avg'].fillna(data['Sales Quantity'].median(), inplace=True)
data['Sales_Monthly_Avg'].fillna(data['Sales Quantity'].median(), inplace=True)

# Label Encoding
le = LabelEncoder()
for col in ['Promotions', 'Seasonality Factors', 'External Factors', 'Demand Trend', 'Customer Segments']:
    data[col] = le.fit_transform(data[col])

# Target variables
threshold = np.percentile(data['Sales Quantity'], 75)
data['High_Demand'] = data['Sales Quantity'].apply(lambda x: 1 if x > threshold else 0)

# Features and Target split
X = data.drop(['Sales Quantity', 'High_Demand', 'Date', 'Product ID', 'Store ID'], axis=1)
y_class = data['High_Demand']
y_reg = data['Sales Quantity']

# Train-test split
X_train, X_test, y_class_train, y_class_test, y_reg_train, y_reg_test = train_test_split(
    X, y_class, y_reg, test_size=0.2, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Classifier
clf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=10, random_state=42)
clf.fit(X_train_scaled, y_class_train)

# Train Regressor
reg = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=10, random_state=42)
reg.fit(X_train_scaled, y_reg_train)

# --- Evaluation ---
y_class_pred = clf.predict(X_test_scaled)
classification_acc = accuracy_score(y_class_test, y_class_pred)

y_reg_pred = reg.predict(X_test_scaled)
reg_rmse = mean_squared_error(y_reg_test, y_reg_pred, squared=False)
reg_r2 = r2_score(y_reg_test, y_reg_pred)

print("🔍 Classification Accuracy:", round(classification_acc, 4))
print("📈 Regression RMSE:", round(reg_rmse, 2))
print("📊 R² Score:", round(reg_r2, 2))

# --- Prediction Function ---
def predict_product_demand_and_sales(product_id):
    current_date = datetime.datetime.now()

    # Get all recent entries for that product
    product_data = data[data['Product ID'] == product_id].sort_values(by='Date', ascending=False)

    if product_data.empty:
        return f"❌ Product ID {product_id} not found."

    results = []
    for _, row in product_data.iterrows():
        X_new = row.drop(['Sales Quantity', 'High_Demand', 'Date', 'Product ID', 'Store ID']).values.reshape(1, -1)
        X_new_scaled = scaler.transform(X_new)

        high_demand_pred = clf.predict(X_new_scaled)[0]
        predicted_sales_quantity = reg.predict(X_new_scaled)[0]
        predicted_month = (row['Date'] + pd.DateOffset(months=1)).strftime('%B %Y')

        result = {
            "Product ID": product_id,
            "Store ID": row['Store ID'],
            "Date": row['Date'].strftime('%Y-%m-%d'),
            "Predicted High Demand (1=Yes, 0=No)": int(high_demand_pred),
            "Target Month": predicted_month,
            "Predicted Sales Quantity": round(predicted_sales_quantity, 2)
        }
        results.append(result)

    return results

# --- Take input from user ---
try:
    user_input = int(input("🔢 Enter Product ID to predict demand and sales quantity: "))
    predictions = predict_product_demand_and_sales(user_input)

    print("\n🎯 Prediction Result(s):")
    if isinstance(predictions, list):
        for res in predictions:
            for k, v in res.items():
                print(f"{k}: {v}")
            print("-" * 40)
    else:
        print(predictions)
except ValueError:
    print("❌ Please enter a valid Product ID.")

🔍 Classification Accuracy: 0.9975
📈 Regression RMSE: 14.59
📊 R² Score: 0.99


🔢 Enter Product ID to predict demand and sales quantity:  4584



🎯 Prediction Result(s):
Product ID: 4584
Store ID: 79
Date: 2024-12-15
Predicted High Demand (1=Yes, 0=No): 1
Target Month: January 2025
Predicted Sales Quantity: 410.13
----------------------------------------
Product ID: 4584
Store ID: 13
Date: 2024-12-06
Predicted High Demand (1=Yes, 0=No): 0
Target Month: January 2025
Predicted Sales Quantity: 330.22
----------------------------------------
Product ID: 4584
Store ID: 93
Date: 2024-12-03
Predicted High Demand (1=Yes, 0=No): 0
Target Month: January 2025
Predicted Sales Quantity: 373.6
----------------------------------------
Product ID: 4584
Store ID: 96
Date: 2024-06-30
Predicted High Demand (1=Yes, 0=No): 1
Target Month: July 2024
Predicted Sales Quantity: 452.95
----------------------------------------
Product ID: 4584
Store ID: 4
Date: 2024-06-03
Predicted High Demand (1=Yes, 0=No): 1
Target Month: July 2024
Predicted Sales Quantity: 431.09
----------------------------------------


In [293]:
predictions = predict_product_demand_and_sales(user_input)


In [295]:
# Convert predictions into a DataFrame
if isinstance(predictions, list):
    pred_df = pd.DataFrame(predictions)
    print("\n📊 Converted Predictions DataFrame:")
    print(pred_df.head())
else:
    print("❌ Could not generate predictions.")



📊 Converted Predictions DataFrame:
   Product ID  Store ID        Date  Predicted High Demand (1=Yes, 0=No)  \
0        4584        79  2024-12-15                                    1   
1        4584        13  2024-12-06                                    0   
2        4584        93  2024-12-03                                    0   
3        4584        96  2024-06-30                                    1   
4        4584         4  2024-06-03                                    1   

   Target Month  Predicted Sales Quantity  
0  January 2025                    410.13  
1  January 2025                    330.22  
2  January 2025                    373.60  
3     July 2024                    452.95  
4     July 2024                    431.09  


In [297]:
inventory_df=pd.read_csv("C://Users//djuid//Downloads//Dataset//Dataset//[use case 1] Inventory Optimization for Retail//inventory_monitoring.csv")

In [299]:
# Step 1: Convert all IDs to strings for consistency
pred_df['Product ID'] = pred_df['Product ID'].astype(str).str.strip()
pred_df['Store ID'] = pred_df['Store ID'].astype(str).str.strip()
inventory_df['Product ID'] = inventory_df['Product ID'].astype(str).str.strip()
inventory_df['Store ID'] = inventory_df['Store ID'].astype(str).str.strip()

# Step 2: Add dummy product data into inventory
new_rows = [
    {"Product ID": "4584", "Store ID": "79", "Stock Levels": 300, "Reorder Point": 350, 
     "Supplier Lead Time (days)": 5, "Stockout Frequency": 2, "Expiry Date": "2025-05-01", 
     "Warehouse Capacity": 500, "Order Fulfillment Time (days)": 3},

    {"Product ID": "4584", "Store ID": "13", "Stock Levels": 320, "Reorder Point": 300, 
     "Supplier Lead Time (days)": 4, "Stockout Frequency": 1, "Expiry Date": "2025-06-01", 
     "Warehouse Capacity": 450, "Order Fulfillment Time (days)": 2},

    {"Product ID": "4584", "Store ID": "93", "Stock Levels": 370, "Reorder Point": 360, 
     "Supplier Lead Time (days)": 6, "Stockout Frequency": 2, "Expiry Date": "2025-07-01", 
     "Warehouse Capacity": 480, "Order Fulfillment Time (days)": 3},

    {"Product ID": "4584", "Store ID": "96", "Stock Levels": 380, "Reorder Point": 400, 
     "Supplier Lead Time (days)": 3, "Stockout Frequency": 2, "Expiry Date": "2025-07-01", 
     "Warehouse Capacity": 470, "Order Fulfillment Time (days)": 2},

    {"Product ID": "4584", "Store ID": "4", "Stock Levels": 350, "Reorder Point": 390, 
     "Supplier Lead Time (days)": 5, "Stockout Frequency": 3, "Expiry Date": "2025-06-25", 
     "Warehouse Capacity": 490, "Order Fulfillment Time (days)": 2},


    
]
# Step 3: Convert list of dicts to a DataFrame and concatenate
new_rows_df = pd.DataFrame(new_rows)
inventory_df = pd.concat([inventory_df, new_rows_df], ignore_index=True)



In [301]:
combined_df = pd.merge(pred_df, inventory_df, on=["Product ID", "Store ID"], how="inner")
print("✅ Merged Rows:", len(combined_df))
print(combined_df.head())


✅ Merged Rows: 5
  Product ID Store ID        Date  Predicted High Demand (1=Yes, 0=No)  \
0       4584       79  2024-12-15                                    1   
1       4584       13  2024-12-06                                    0   
2       4584       93  2024-12-03                                    0   
3       4584       96  2024-06-30                                    1   
4       4584        4  2024-06-03                                    1   

   Target Month  Predicted Sales Quantity  Stock Levels  \
0  January 2025                    410.13           300   
1  January 2025                    330.22           320   
2  January 2025                    373.60           370   
3     July 2024                    452.95           380   
4     July 2024                    431.09           350   

   Supplier Lead Time (days)  Stockout Frequency  Reorder Point Expiry Date  \
0                          5                   2            350  2025-05-01   
1                        

In [303]:
def calculate_reorder(row):
    if row['Stock Levels'] < row['Predicted Sales Quantity']:
        reorder_qty = row['Predicted Sales Quantity'] - row['Stock Levels']
        return max(reorder_qty, 0)
    else:
        return 0  # no need to reorder

combined_df['Reorder Quantity'] = combined_df.apply(calculate_reorder, axis=1)


In [305]:
final_df = combined_df[[
    'Product ID', 'Store ID', 'Stock Levels', 'Predicted Sales Quantity', 'Reorder Quantity'
]]
print("\n✅ Final Decision Table:")
print(final_df)



✅ Final Decision Table:
  Product ID Store ID  Stock Levels  Predicted Sales Quantity  \
0       4584       79           300                    410.13   
1       4584       13           320                    330.22   
2       4584       93           370                    373.60   
3       4584       96           380                    452.95   
4       4584        4           350                    431.09   

   Reorder Quantity  
0            110.13  
1             10.22  
2              3.60  
3             72.95  
4             81.09  
